<a href="https://colab.research.google.com/github/klesment/PopProj/blob/main/PopulationProjection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Population projection exercise

### Cohort-component method with adjustable fertility scenarios

Cohort-component method of population projection is a deterministic way to predict how the age composition and size of population change as time passes. Since future fertility, migration flows, and mortality rates are unknown, projections employ scenarios. For example, projecting the population of a low fertility country, one could expect that the future fertility stays the same, increases over the projection period, or even declines.  

In the following, we produce a projection that allows the user to change the fertility scenario and manipulate the parameters of the future age-specfic fertility rates (mean age at birth and its dispersion). In addition, we allow the user to change how fast fertility changes over the projected period (ramp speed). For the sake of simplicity, scenarios will be applied only to fertility, while mortality rates are kept constant and international migration is not taken into account.

### Data

To initiate a projection, we need data about current population processes. In this exercise, we will use [Estonian](https://en.wikipedia.org/wiki/Estonia) data, which, among other things, will exemplify the importance of [age structure](https://www.stat.ee/rahvastikupyramiid/?lang=en) for fertility.  

We will base the calculation on the following information from the year 2019:

* population size by 1-year age groups
* age-specific mortality rates (from life table)
* age specific fertility rates by 1-year age groups
* mean age at birth
* standard deviation of mean age at birth

All data are sourced from: [Human fertility DB](http://humanfertility.org) and [Human mortality DB](http://mortality.org). Since both domains require account login to access data, the files have been pre-downloaded and stored on GitHub.

In [14]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import gamma
import requests
import re
import io
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import interact, interact_manual, interactive, fixed, IntSlider, FloatSlider, HBox, Layout, VBox
import cufflinks as cf
cf.go_offline()

from IPython.display import Markdown as md
%matplotlib inline

In [15]:
# Data files and constant values

# age specific fertility rates
URL = 'https://raw.githubusercontent.com/klesment/PopProj/main/ESTasfrRR.txt'
s = requests.get(URL).content
asfr = pd.read_csv(io.StringIO(s.decode('utf-8')), delim_whitespace=True, header = 2)
asfr = asfr.apply(pd.to_numeric, errors = 'coerce')

# life table data
URL = 'https://raw.githubusercontent.com/klesment/PopProj/main/LT.txt'
s = requests.get(URL).content
lt = pd.read_csv(io.StringIO(s.decode('utf-8')), delim_whitespace=True, header = 1)
lt.loc[lt['Age'] == '110+', 'Age'] = 110 # open upper age interval to 110
lt = lt.apply(pd.to_numeric)
lt.loc[lt['qx'] == 0, 'qx'] = np.nan # zeros to NaN

# population counts
URL = 'https://raw.githubusercontent.com/klesment/PopProj/main/Population.txt'
s = requests.get(URL).content
pop = pd.read_csv(io.StringIO(s.decode('utf-8')), delim_whitespace=True, header = 1)
pop.loc[pop['Age'] == '110+', 'Age'] = 110 # open upper age interval to 110
pop = pop.apply(pd.to_numeric)

# mean age at birth in 2019
mab = 30.62

# standard deviation of mean age at birth in 2019
sd_mab = 5.62

## Input values

### Mortality
To project population from one year to the next, we need to know the probability of surviving at each age. This information comes from the life table and these values are kept constant over the projected period. Moreover, we do not distinguish men and women, but use the 2019 life table of total population.

Here are the first three lines of our life:

In [16]:
lt[lt['Year']==2019].head(3)

,Year,Age,mx,qx,ax,lx,dx,Lx,Tx,ex
6660,2019,0,0.00087,0.00087,0.15,100000,87,99926,8273542,82.74
6661,2019,1,0.00014,0.00014,0.50,99913,14,99906,8173616,81.81
6662,2019,2,0.00015,0.00015,0.50,99899,15,99891,8073710,80.82


#### A short note on past mortality changes
As our life table data frame actually covers years since 1959, we can here briefly demonstrate how mortality and life expectancy have changed in the past. In our exercise we assume constant mortality (as of 2019) over the projected period.

Below, an interactive figure allows a comparison of a select year's age-specific mortality (probability of dying) with the 2019 data. Figure on the right shows the improvement in life expectancy over the past period that is covered by life tables.

In [17]:
plt.rcParams['figure.figsize'] = [12, 6]

def mx_ex_plot(m):
    fig, ax = plt.subplots(1,2)
    
    p = sns.lineplot(data=m, x='Age', y = 'qx', hue = 'Year', palette=["C0", "C1"], ax = ax[0]); p.set_yscale('log')
    p.set_title("Probability of dying (log-scale)")

    o = sns.lineplot(x=range(1958,2019), y = lt[lt['Age']==0]['ex'], ax = ax[1]); o.set(xlabel='Year', ylabel='ex')
    o.set_title("Life expectancy 1959-2019")

@interact
def show_mx_ex_plot(Year = range(1959,2019,5)):
    z = lt.loc[lt['Year'].isin([Year, 2019])]
    mx_ex_plot(z)

interactive(children=(Dropdown(description='Year', options=(1959, 1964, 1969, 1974, 1979, 1984, 1989, 1994, 19…



---



---



### Fertility

In the projection, the number of newborns is determined by age-specific fertility rate (ASFR). ASFR is a ratio, where the numerator is the annual number of births to women who are of certain age or age group, and the denominator is the number of person-years lived by  women of the same group. In this case we use ASFR that is calculated for each single year of age between 12 and 55. 

Sum of ASFRs over the age range produces the year's Total Fertility Rate (TFR), which is the most common indicator of period fertility. 

In the figure below we allow users to compare ASFR of a selected year to ASFR of 2019. The plot also shows the sum of these rates, i.e. TFR.

In [18]:
plt.rcParams['figure.figsize'] = [12, 6]

def asfr_plot(t):
    p = sns.lineplot(data=t, x="Age", y="ASFR", hue="Year", palette=["C0", "C1"])
    p.set(ylim=(0, 0.2))
    p.annotate(f"{min(t['Year'])} Total Fertility Rate: {round(sum(t['ASFR'][t['Year']!=2019]),2)}", xy=(40,0.11), size=12)
    p.annotate(f"{max(t['Year'])} Total Fertility Rate: {round(sum(t['ASFR'][t['Year']==2019]),2)}", xy=(40,0.09), size=12)

@interact
def asfr_comparison(Year=(1959, 2018, 1)):
    z = asfr.loc[asfr['Year'].isin([Year, 2019])]
    asfr_plot(z)

interactive(children=(IntSlider(value=1988, description='Year', max=2018, min=1959), Output()), _dom_classes=(…



---



---



## Projection using Leslie matrix


[Leslie matrix](https://en.wikipedia.org/wiki/Leslie_matrix) is a method to project populations based on survival probabilities and fertility rates. 



We use the following notation:

$x_0, x_1 ... x_n$ - single year age-groups where $n$ goes from 0 and 110. 

$t_0, t_m$ - time to project in $m$ calendar years, while $t_0$ is the year from which projection starts

$f_1, f_2 ... f_x$ - age-specific fertility rates

$P_1, P_2 ... P_x$ - age-specific probabilities of surviving




An example of a Leslie matrix with only three age groups:


\begin{equation*}
\mathbf{L}=%
\begin{pmatrix}
f_{1} & f_{2} & f_{3} \\ 
P_{1} & 0  & 0 \\ 
0 & P_{3} & 0% 
\end{pmatrix}%
\end{equation*}


If the age structure of the base year (vector $x$) is multiplied with the matrix $\mathbf{L}$ (which has the same side length as $x$), it results in a new  $t_{0+1}$ shifted age structure $x_t$, in which the first element of the vector is the number of newborns: 


\begin{equation*}
x_t =
\begin{pmatrix}
f_{1} & f_{2} & \cdots & f_{x} \\ 
P_{1} & 0 & \cdots & 0 \\ 
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & P_{x-1} & P_x
\end{pmatrix} 
\begin{pmatrix}
           x_{1} \\
           x_{2} \\
           \vdots \\
           x_{n}
         \end{pmatrix}
\end{equation*}


Such projection can also be carried out over a longer period than 1 year. In that case, the lenght of the projected period becomes the exponent of $\mathbf{L}$ while it is multiplied with the base year's age structure.

\begin{equation*}
x_t = \mathbf{L}^t x_0
\end{equation*}

In our case, however, we employ repeated single-year projections, since fertility rates in the matrix will change each year.

### Example projection with fixed fertility rates

To illustrate how Leslie matrix works, we first use it to project a population using fixed fertility rates. This means that the rates will stay the same over the projected period as they are in the base year of 2019.

Below, we first construct the matrix and populate it with survival probabilities and fertility rates. 

Then, the function `'proj'` that performs projection is created. 

In [19]:
# Constructing and populating the Leslie matrix

# starting values
base_year = 2019

# life table
lt19 = lt[lt['Year']==base_year]

# population structure
pop19 = pop[pop['Year']==base_year]
N0 = pop19.Total[0:110].tolist()

# ASFR
asfr19 = asfr[asfr['Year']==base_year]

# Matrix elements
# Probability of survival at each age from life table, goes to matrix subdiagonal
SUBD = lt19.Lx.iloc[1:].values/lt19.Lx.iloc[0:110].values
SUBD[-1:] = 0
Tx = lt19['Tx']

# Age-specific fertility rates, values for ages 12-54
Fx = [0]*12 + asfr19.ASFR.values.tolist() + [0]*55
k = (1/(1+1.05))*(lt19.Lx.iloc[0]/200000)
R1 = (k*(np.array(Fx[0:110]) + np.array(Fx[1:111]) * np.array(SUBD[0:110])))

# matrix
L = np.zeros((109, 110))

np.fill_diagonal(L, SUBD[0:109])

L = np.vstack((R1,L))

L[109,109] = Tx.values[110]/Tx.values[109]


# Projection function

def proj(m, t, n0):
    Nproj = []
    
    for i in range(0, t, 1):
        if i == 0:
            project = m
            Nproj = np.dot(project, n0)
        
        if i > 0:
            project = np.dot(project, m)
            Nproj = np.dot(project, n0)
                
    return Nproj

Next, we create an interactive plot that allows users to choose the length of projection, while projected values are generated for every 25th year since the start of projection. 

This plot is basically a sideways population pyramid, with age on the x-axis, but where gender is not differentiated.

As shown in the graph, keeping the ASFR of 2019 constant over the projected years will result in a substantial reduction of annual number of births (from around 15 thousand in 2020 to around 12 thousand in 2045), which is due to the changing age structure of people in childbearing age.

In [20]:
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = [12, 6]

def proj_graph(time):
    for i in range(1, time, 25):
        y_data = proj(L, i, N0)
        p = sns.lineplot(x = range(1,111), y = y_data, label=str(i+base_year))
        p.legend(title='Year')
        p.set(xlabel='1-year age group', ylabel='Count')

@interact
def proj_sliders(
    Period=(10, 100, 10)):
    proj_graph(Period)

interactive(children=(IntSlider(value=50, description='Period', min=10, step=10), Output()), _dom_classes=('wi…



---



---





---


## Projection with dynamic fertility rates

To improve on the constant fertility scenario, we want to change the projection so that ASFR would change over the projected period, as we expect TFR to rise or decline. This means, we need to update ASFR at each cycle of projection.  

Of course, we do not know future ASFR, but we can estimate what it could be based on our expected scenario of total fertility rate and mean age at birth.

ASFR can be modelled with a $\Gamma$-model which has three parameters: TFR, mean age at birth (MAB) and the standard deviation of mean age at birth (sdMAB). 

The output of the $\Gamma$-model is the fertility rate $F$ at age $x$, on the condition that that $x$ is higher than the parameter $\alpha_4$.  


\begin{equation*}
F_x = \frac{1}{\Gamma (\alpha_3)} \alpha_1 \alpha_2^{\alpha_3}(x - \alpha_4)^{\alpha_3 - 1} exp(-\alpha_2(x - \alpha_4)); x \ge \alpha_4
\end{equation*}

In practice, $\alpha_4$ is taken as equal to 0 and other model parameters are defined as follows: 

* $\alpha_1 = TFR$

* $\alpha_2 = \frac{MAB}{sdMAB^2}$

* $\alpha_3 = (\frac{MAB}{sdMAB})^2$

The model needs following inputs: 

* TFR - total fertility rarte

* MAB - mean age at birth

* sdMAB - dispersion of mean age at birth.

These three will be user inputs in our projection.

In [21]:
# Gamma model function

def asfr_gamma(aa):
    f_x_out = []

  # parameters
    a1 = aa['tfr'] 
    a2 = aa['mab']/(aa['sd_mab']**2)
    a3 = (aa['mab']/aa['sd_mab'])**2
  
  # age range
    ages = list(range(12,50,1))

    for age in ages:
        v = (1/gamma(a3))*a1*(a2**a3)*(age**(a3-1))*np.exp(-a2*age)
        f_x_out.append(v)

  # 1-D array
    return np.array(f_x_out).ravel()

### Leslie projection function with dynamic fertility rates

We use the previously defined `'proj'` function in a new loop function `'proj_dyn'`. Additionally, we create another function `'leslie'` which will be used in the projection to populate the matrix. 

For the $Γ$-model to work, we need also user input values: change in TFR, MAB and sdMAB at the end of the projected period. The values between the start of projection and end of projection will be linearly interpolated. The user will provide these values for the end of the projection and the model will use the interpolated values for each year between the start and end of the projection.

In [22]:
# Define new functions

# dynamic projection function

def project_dyn(lmat, pop, per):
    '''
    lmat - yearly L matrix; pop - initial population structure; per - period to be projected
    '''

    N_0 = pop

    for i in range(1, per, 1):
        N_x = proj(lmat[(i-1)], 1, N_0)
        N_0 = N_x
  
    return N_0


# Leslie matrix function

def leslie(fert):
    '''
    fert - ASFR vector; 
    mort - life table dataframe
    'lt19' data frame must be present
    '''
    mort = lt19
  
    SUBD = mort.Lx.iloc[1:].values/mort.Lx.iloc[0:110].values
    SUBD[-1:] = 0

    Tx = mort['Tx']

    Fx = [0]*12 + fert.values.tolist() + [0]*61
    k = (1/(1+1.05))*(mort.Lx.iloc[0]/200000)
    R1 = (k*(np.array(Fx[0:110]) + np.array(Fx[1:111]) * np.array(SUBD[0:110])))

    L = np.zeros((109, 110))
    np.fill_diagonal(L, SUBD[0:109])
    L = np.vstack((R1,L))
    L[109,109] = Tx.values[110]/Tx.values[109]

    return L

### Ramp function

To add another dynamic function to the projection, we may want to change the speed at which the TFR changes. For example, we may assume that the change is not happening evenly over the period but is concentrated in the beginning of the projected period. 

To this end, we add a 'ramp function' to the projection. The higher the value, the faster the increase or decrease in TFR is expected to happen. 

In [23]:
# TFR change ramp function

def ramp_fun(TFR_chng,speed,pr_per):
    a,b,c = TFR_chng,5,speed 

    x = np.linspace(0,1,pr_per)
    y = a*np.exp(-b*np.exp(-c*x))
    y2 = [y[-1]]*0
    return np.add([*y,*y2],1)

In [24]:
# ramp function example

'''
A - proportion of total increase/decrease over the period
B - ramp slope
'''

def curve(a,c):
    p2,a,b,c = 100,a,5,c

    x = np.linspace(0,1,p2)
    y = a*np.exp(-b*np.exp(-c*x))
    y2 = [y[-1]]*0
    plt.plot(np.add([*y,*y2],1))
    plt.xlabel('Projected years', fontsize=14)
    plt.ylabel('Proportional change', fontsize=14)

@interact
def sig_sliders(
    A=(-.5, .5, 0.1),
    B=(5, 8, .1)):
    curve(A, B)

interactive(children=(FloatSlider(value=0.0, description='A', max=0.5, min=-0.5), FloatSlider(value=6.0, descr…



---



---



## Final result, interactive projection plot

In the last cell, we generate the interactive projection plot. There are 5 sliders for user input: 

1.   `TFR_Change` proportion of change in TFR by the end of the projection 
2.   `Ramp` - ramp speed of TFR change (varies from 5 to 8)
3.   `MAB_end` - mean age at birth at the end of projection
4.   `sdMAB_end` - standard deviation of MAB at the end of projection
5.   `Years` - length of projection in years (in 5-year steps)

When generating the plot, the values will first be set as: 

*   10% increase in TFR
*   minimum ramp speed (almost linear increase)
*   mean age at birth 30 years
*   projection lenght 25 years. 


The dotted *red* vertical line on the graph indicates the cohort that was born in 2019. That is, all ages to left of the red line in the graph are projected. 

At the top of the graph, initial (2019) values of TFR, mean age at birth and total size of population are shown. 

At the bottom of the graph, the same is shown for the last year of the predicted period. 

There also two annotations in red: $x_0^t$ is the number of newborns in the last year of the predicted period. Pop.% is the size of the total population compared with the 2019 size in percentages. 

In [26]:
plt.rcParams['figure.figsize'] = [14, 8]

def dynamic_graph(TFR_Change, Ramp, MAB_end, sdMAB_end, Years):
    mab_stop,sd_mab_stop,period = MAB_end,sdMAB_end,Years
  
  # fixed starting values from source data frames
    tfr_start,mab_start,sd_mab_start = round(sum(asfr['ASFR'][asfr['Year']==base_year]),2),mab,sd_mab

    if period==0:
        period += 1

  # user input values
    d = pd.DataFrame(data={
        'tfr': np.linspace(tfr_start, tfr_start, period),
        'mab': np.linspace(mab_start, mab_stop, period),
        'sd_mab': np.linspace(sd_mab_start, sd_mab_stop, period)
     })

    d['tfr'] = np.multiply(d['tfr'], ramp_fun(TFR_Change,Ramp,period))
  
    if period==0:
        tfr_last = round(d['tfr'][period],2)
    if period>0:
        tfr_last = round(d['tfr'][period-1],2)

  # age-specific fertility rates from the gamma function
    F = []
    F = np.array(d.apply(asfr_gamma, axis=1).tolist())

    D = pd.DataFrame(data=F[0:,0:])

  # Leslie matrices as lists
    LL = np.array(D.apply(leslie, axis=1))
  
  # projection function
    out = project_dyn(LL, N0, period)

    p_size_start,p_size_end = round(sum(pop['Total'][pop['Year']==base_year])/1000_000,3),round(sum(out)/1000000, 3)

    p = sns.lineplot(x=range(1,111,1), y=out, linewidth=2.5)
    if period<110:
        plt.plot([period,period], [5000,15000], color='red', ls='dotted')
        p.set(ylim=(0, 25_000))
        p.set(xlabel='1-year age group', ylabel='Count')
        p.annotate(f"{base_year + len(LL)}: TFR={tfr_last} mAB={mab_stop} N={p_size_end} Mio.", xy=(0,1_000), size=16)
        p.annotate(f"{base_year}: TFR = {tfr_start} mAB={mab_start} N={p_size_start} Mio.", xy=(0,23_000), size=16)
        p.annotate(f"$x_0^t$: {round(out[0])}", xy=(80,22_000), color="red", size=16)
        p.annotate(f"Pop.%: {round(p_size_end/p_size_start*100,1)}", xy=(80,16_000), color="red", size=16)
        

widget = interactive(dynamic_graph,
                TFR_Change = FloatSlider(min=-.5, max=.5, step=.1, value=0.1, orientation='horizontal'),
                Ramp = FloatSlider(min=5, max=8, step=.1, value=5, orientation='horizontal'),
                MAB_end=IntSlider(min=18, max=36, step=1, value=30, orientation='horizontal'),
                sdMAB_end=FloatSlider(min=5, max=6, step=.1, value=5.5, orientation='horizontal'),
                Years=IntSlider(min=0, max=200, step=5, value=25, orientation='horizontal'))

controls = HBox(widget.children[:-1], layout = Layout(flex_flow='row wrap'))
output = widget.children[-1]
display(VBox([controls, output]))
widget.update()



---



---



End of exercise.

# Thanks for your attention!

---



---

